In [ ]:
import cv2
import glob,os
import numpy as np
from tqdm import tqdm
import json
from myutil import *

from mydataloader import FrameProcessor, YoloLoader, PoseModelLoader

class myVideoLoader(object):
    def __init__(self, videoPath):
        self.videoPath = videoPath
        self.dirPath , self.videoname = os.path.split(videoPath)
        self.cap = cv2.VideoCapture(videoPath)
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS)) 
        self.fcount = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    def get_audio(self):
        cap = cv2.VideoCapture(self.videoPath)
        ret, frame = cap.read()
        self.audioPath = self.videoPath[:-4]+'.wav'
        command = "ffmpeg -i {} -vn -ar 16000 -f wav {}".format(self.videoPath, self.audioPath)
        try:
            subprocess.check_call(command, shell=True)
            is_success = True
        except subprocess.CalledProcessError as e:
            print("error code: {}! shell command: {}".format(e.returncode, e.cmd))
            is_success = False
        assert is_success==True, print('convert error!')
        cap.release()
        return self.audioPath
     
    
    def extract_frame(self, savDir, by_sec=1, useFilter = True, min_det_num = 40, filter_jump_sec=5):
        #by_sec=5 : save frame every 5 second 
        #useFilter, min_det_num : use alphapose to detect person number and filter the frame (detect_num<min_det_num)
        #filter_jump_sec : if detect frame, jump x sec.
        if not os.path.exists(savDir):
            os.makedirs(savDir)
            
        if useFilter:
            # Load model
            pose_model_loader = PoseModelLoader()
            det_model_loader = YoloLoader()
            
        cap = cv2.VideoCapture(self.videoPath)
        fI_jump=0
        for fI in tqdm(range(0,self.fcount,self.fps*by_sec)):
            if fI<fI_jump:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            if useFilter:
                framePose = FrameProcessor(frame, det_model_loader, pose_model_loader).start()
                result = framePose.resultNew
                person_num = framePose.det_human_num
                if person_num<min_det_num:
                    continue
            imgSavname=self.videoname[:-4]+'_'+str(fI)+'_'+str(person_num)+'.jpg'
            jsonSavname=self.videoname[:-4]+'_'+str(fI)+'_'+str(person_num)+'.json'
            sav_img(os.path.join(savDir,self.videoname[:-4],imgSavname), frame)
            with open(os.path.join(savDir,self.videoname[:-4],jsonSavname),'w',encoding='utf-8') as jsonfile:
                json.dump(result, jsonfile)
            fI_jump = fI + self.fps*filter_jump_sec
            #print(person_num, savname)
        cap.release() 
    
    def auto_extract_frame(self, savDir, by_sec=1, filter_jump_sec=5):
        imgSavDir = os.path.join(savDir,self.videoname[:-4],'images')
        jsonSavDir = os.path.join(savDir,self.videoname[:-4],'keypoints')
        
        if not os.path.exists(savDir):
            os.makedirs(savDir)
        if not os.path.exists(imgSavDir):
            os.makedirs(imgSavDir)
        if not os.path.exists(jsonSavDir):
            os.makedirs(jsonSavDir)
        
        # Load model
        pose_model_loader = PoseModelLoader()
        det_model_loader = YoloLoader()
            
        cap = cv2.VideoCapture(self.videoPath)
        fI_jump=0
        fIList=[]
        personNumList=[]
        resultList=[]
        for fI in tqdm(range(0,self.fcount,self.fps*by_sec)):
            #if fI<fI_jump:
                #continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            framePose = FrameProcessor(frame, det_model_loader, pose_model_loader).start()
            result = framePose.resultNew
            personNum = framePose.det_human_num
            personNumList.append(personNum)
            resultList.append(result)
            fIList.append(fI)
            #fI_jump = fI + self.fps*filter_jump_sec
        
        personNumThre=int(0.9*max(personNumList))
        for fI, personNum, result in tqdm(zip(fIList, personNumList, resultList)):
            if personNum<personNumThre:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            imgSavName=self.videoname[:-4]+'_'+str(fI)+'_'+str(personNum)+'.jpg'
            jsonSavName=self.videoname[:-4]+'_'+str(fI)+'_'+str(personNum)+'_keypoints.json'
            sav_img(os.path.join(imgSavDir, imgSavName), frame)
            with open(os.path.join(jsonSavDir, jsonSavName),'w',encoding='utf-8') as jsonfile:
                json.dump(result, jsonfile)
        cap.release() 

videoDir=r'/home/ubuntu/Data/Video/class/'
savDir='/home/ubuntu/Data/autoFrameFilter/'
videoPathList = glob.glob(videoDir+'*.mp4')
for videoPath in videoPathList:
    testVideo = myVideoLoader(videoPath)
    testVideo.auto_extract_frame(savDir,by_sec=1)